# Resumo

Neste notebook, o método de Grid Search é aplicado à uma Rede Neural para o ajuste de parâmetros.<br>

<br>
O dataset utilizado é o Breast Cancer Winsconsin, do repositório UC Irvine.<br>
Possui 569 amostras, cada uma com 32 variáveis (1 ID, 1 target e 30 atributos).<br>
Target: 0 para begnino, 1 para maligno (classificação binária).<br>
Não há dados faltantes.<br>

# Importação dos recursos

In [1]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV # classe com funcionalidade para implmentação de Grid Search com Cross Validation
from scikeras.wrappers import KerasClassifier

2024-08-21 18:08:05.855502: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X = pd.read_csv('Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/entradas_breast.csv')
y = pd.read_csv('Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/saidas_breast.csv')

# Criação da Rede Neural

In [13]:
def create_network(neurons, activation, kernel_initializer, optimizer, loss):
    k.clear_session() #limpa a sessão do TensorFlow
    network = Sequential([
        tf.keras.layers.InputLayer(shape=(30,)),
        tf.keras.layers.Dense(units = neurons, activation=activation, kernel_initializer=kernel_initializer),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = neurons, activation=activation, kernel_initializer=kernel_initializer),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(units = 1, activation='sigmoid') # precisa ser Sigmoid, por se tratar de classificação binária
    ])
    network.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy']) # a métrica é fixa pois Binary Accuracy é o método ideal para classificações binárias
    return network

In [14]:
network = KerasClassifier(model = create_network)

# Configuração da Grade de Parâmetros

In [15]:
parameters = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__neurons': [16],
    'model__activation': ['relu'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy']
}

Quantidade de execuções do treinamento:<br>
2 * 1 * 1 * 1 * 2 * 1 * 1 * 5 = 20<br>
OBS: o 10 é o número de Folds na Cross Validation, especificado abaixo:

In [16]:
grid_search = GridSearchCV(estimator = network, param_grid=parameters, scoring='accuracy', cv = 5)

In [17]:
grid_search = grid_search.fit(X, y)

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - binary_accuracy: 0.4612 - loss: 4.5136
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5649 - loss: 0.6883
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5983 - loss: 0.6530
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6946 - loss: 0.5454
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6932 - loss: 0.5084
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7214 - loss: 0.5157
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7494 - loss: 0.4973
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7277 - loss: 0.4872
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7247 - loss: 0.4689
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7649 - loss: 0.4519
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8413 - loss: 0.42

In [19]:
print(grid_search)

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function create_network at 0x7de0827f87c0>),
             param_grid={'batch_size': [10, 30], 'epochs': [50],
                         'model__activation': ['relu'],
                         'model__kernel_initializer': ['random_uniform',
                                                       'normal'],
                         'model__loss': ['binary_crossentropy'],
                         'model__neurons': [16], 'model__optimizer': ['adam']},
             scoring='accuracy')


In [20]:
best_params = grid_search.best_params_
best_params

{'batch_size': 30,
 'epochs': 50,
 'model__activation': 'relu',
 'model__kernel_initializer': 'random_uniform',
 'model__loss': 'binary_crossentropy',
 'model__neurons': 16,
 'model__optimizer': 'adam'}

Esses foram os melhores valores para os parâmetros da rede neural encontrados pela Grid Search.

In [21]:
best_precision = grid_search.best_score_
best_precision

0.8945660611706255

A melhor configuração encontrada para a Rede Neural gera uma precisão de 0.89, superior ao que foi nas configurações, feitas manualmente, testadas anteriormente em outros notebooks.

## Versão completa (mais tempo de treinamento):

In [23]:
parametersComplete = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'model__neurons': [16, 8],
    'model__activation': ['relu', 'tanh'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge']
}

Quantidade de execuções do treinamento:<br>
2 * 2 * 2 * 2 * 2 * 2 * 2 * 10 = 5<br>
OBS: o 5 é o número de Folds na Cross Validation. Na prática, se usa pelo menos 10, mas por questões de tempo serão utilizar apenas 5 Folds neste teste.

In [24]:
grid_search2 = GridSearchCV(estimator = network, param_grid=parametersComplete, scoring='accuracy', cv = 5)

In [ ]:
grid_search2 = grid_search2.fit(X, y)

A Grid Search com parametersComplete não foi executada por falta de tempo, mas toda a configuração está pronta, bastaria executar a linha acima.